In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
file_path = "./drive/MyDrive/annot.TextGrid"
#file_path = "./drive/MyDrive/annot_item1.TextGrid"
print(os.path.exists(file_path))

True


In [5]:
DATA_PATH = "./drive/MyDrive"  # if the data uploaded to the google drive.
video_path = os.path.join(DATA_PATH, "nagsync.mp4")

In [6]:
import chardet

# Detect file encoding
with open(file_path, "rb") as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding = result['encoding']

print(f"Detected Encoding: {encoding}")

# Now try reading the file with the detected encoding
with open(file_path, 'r', encoding=encoding) as file:
    content = file.read()
    print(content[:500])  # Print first 500 characters to check if it loads correctly


Detected Encoding: UTF-16
File type = "ooTextFile"
Object class = "TextGrid"

xmin = 0 
xmax = 361.3015873015873 
tiers? <exists> 
size = 2 
item []: 
    item [1]:
        class = "IntervalTier" 
        name = "1" 
        xmin = 0 
        xmax = 361.3015873015873 
        intervals: size = 191 
        intervals [1]:
            xmin = 0 
            xmax = 11.210359544496306 
            text = "" 
        intervals [2]:
            xmin = 11.210359544496306 
            xmax = 11.705565537272031 
            text =


In [8]:
import chardet
import json

def parse_textgrid(file_path):
    # Detect encoding
    with open(file_path, "rb") as f:
        rawdata = f.read()
        encoding = chardet.detect(rawdata)['encoding']

    with open(file_path, 'r', encoding=encoding) as file:
        textgrid_content = file.readlines()

    word_intervals = []
    letter_intervals = []
    current_interval = {}
    current_tier = None

    for line in textgrid_content:
        line = line.strip()

        # Detecting tiers (item[1] for words, item[2] for letters)
        if line.startswith("item [1]:"):
            current_tier = "words"
        elif line.startswith("item [2]:"):
            current_tier = "letters"

        if line.startswith("xmin ="):
            current_interval["xmin"] = float(line.split("=")[1].strip())

        elif line.startswith("xmax ="):
            current_interval["xmax"] = float(line.split("=")[1].strip())

        elif line.startswith("text ="):
            text_value = line.split("=")[1].strip().strip('"')
            current_interval["text"] = text_value

            if text_value:
                if current_tier == "words":
                    word_intervals.append(current_interval)
                elif current_tier == "letters" and text_value in {"p", "b", "f", "v"}:
                    letter_intervals.append(current_interval)

            current_interval = {}  # Reset for the next interval

    return word_intervals, letter_intervals

def match_letters_to_words(word_intervals, letter_intervals):
    """Match letters (p, b, f, v) to their corresponding word intervals."""
    output_data = {}

    for i, word in enumerate(word_intervals):
        word_text = word["text"]
        word_xmin = word["xmin"]
        word_xmax = word["xmax"]

        # Find letters that fall within the word's time interval
        matched_letters = [
            (letter["text"], letter["xmin"], letter["xmax"])
            for letter in letter_intervals
            if word_xmin <= letter["xmin"] <= word_xmax
        ]

        # Select the first matching letter (assuming only one occurs per word)
        if matched_letters:
            letter_text, Lmin, Lmax = matched_letters[0]
        else:
            letter_text, Lmin, Lmax = None, None, None  # No matching letter

        output_data[i] = {
            "word": word_text,
            "xmin": word_xmin,
            "xmax": word_xmax,
            "letter": letter_text,
            "Lmin": Lmin,
            "Lmax": Lmax
        }

    return output_data

# Example Usage
textgrid_file = file_path
word_intervals, letter_intervals = parse_textgrid(textgrid_file)
output_dict = match_letters_to_words(word_intervals, letter_intervals)

# Save as JSON
output_file = "parsed_textgrid_with_letters.json"
with open(output_file, "w", encoding="utf-16") as f:
    json.dump(output_dict, f, indent=4, ensure_ascii=False)

print(f"Output saved as {output_file}")


Output saved as parsed_textgrid_with_letters.json


In [9]:
output_dict

{0: {'word': 'kukbu',
  'xmin': 11.210359544496306,
  'xmax': 11.705565537272031,
  'letter': 'b',
  'Lmin': 11.526556747509579,
  'Lmax': 11.542752921158321},
 1: {'word': 'kukbu',
  'xmin': 13.166530580509903,
  'xmax': 13.644888604681785,
  'letter': 'b',
  'Lmin': 13.458892089293718,
  'Lmax': 13.4752442509548},
 2: {'word': 'vakit',
  'xmin': 15.450812804444787,
  'xmax': 15.98259808429414,
  'letter': 'v',
  'Lmin': 15.48103235387536,
  'Lmax': 15.517079737886517},
 3: {'word': 'kufkak',
  'xmin': 18.424850799035156,
  'xmax': 19.00027588650754,
  'letter': 'f',
  'Lmin': 18.566248902810408,
  'Lmax': 18.624603675797022},
 4: {'word': 'kakbu',
  'xmin': 21.88719862383491,
  'xmax': 22.41285843977225,
  'letter': 'b',
  'Lmin': 22.131749907127606,
  'Lmax': 22.24673689133878},
 5: {'word': 'kafuka',
  'xmin': 25.64465988647007,
  'xmax': 26.102695315292678,
  'letter': 'f',
  'Lmin': 25.776730025731588,
  'Lmax': 25.860260036876056},
 6: {'word': 'kafka',
  'xmin': 29.102218026482

In [ ]:
import re

def parse_textgrid(file_path):
    with open(file_path, 'r', encoding='utf-16') as file:
        textgrid_content = file.readlines()

    intervals = []
    current_interval = {}

    for line in textgrid_content:
        line = line.strip()

        if line.startswith("xmin ="):
            current_interval["xmin"] = float(line.split("=")[1].strip()) * 2

        elif line.startswith("xmax ="):
            current_interval["xmax"] = float(line.split("=")[1].strip()) * 2

        elif line.startswith("text ="):
            text_value = line.split("=")[1].strip().strip('"')
            current_interval["text"] = text_value

            if text_value:  # Only add if text is not empty
                intervals.append(current_interval)

            current_interval = {}  # Reset for the next interval

    result_dict = {i: [interval["text"], interval["xmin"], interval["xmax"]] for i, interval in enumerate(intervals)}

    return result_dict

# Example usage:
# file_path = "annot.TextGrid"
output_dict = parse_textgrid(file_path)
print(output_dict)


{0: ['kukbu', 22.420719088992612, 23.411131074544063], 1: ['kukbu', 26.333061161019806, 27.28977720936357], 2: ['vakit', 30.901625608889574, 31.96519616858828], 3: ['kufkak', 36.84970159807031, 38.00055177301508], 4: ['kakbu', 43.77439724766982, 44.8257168795445], 5: ['kafuka', 51.28931977294014, 52.205390630585356], 6: ['kafka', 58.2044360529657, 59.205928538009786], 7: ['kupku', 65.19901463515454, 66.1204541216079], 8: ['tapu', 74.14359798467221, 74.88476857016809], 9: ['ufuk', 81.99970895392961, 82.9179422997029], 10: ['kuvuk', 91.87948264194883, 92.79641068939095], 11: ['kabuk', 107.85519996605454, 108.78702171767685], 12: ['kukbak', 114.83810151829385, 116.00731617013003], 13: ['kufkak', 122.51740058586829, 123.74161480007548], 14: ['paket', 130.13204996687705, 131.17286864168366], 15: ['afkun', 137.08357221029226, 138.18807516712894], 16: ['bacak', 143.16496852882608, 144.0930411834269], 17: ['kavun', 150.09509683537752, 150.9483656856318], 18: ['kafkuk', 161.08157521227523, 162.

In [ ]:
import json
# Save the output dictionary as a JSON file
output_file = "parsed_textgrid.json"  # DETERMINE THE NAME OF THE JSON FILE
with open(output_file, "w", encoding="utf-16") as f:  # Initial tests tells us it gives utf-16
    json.dump(output_dict, f, indent=4, ensure_ascii=False)

print(f"Output saved as {output_file}")

Output saved as parsed_textgrid.json


# Cutting the Videos

In [ ]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
!pip install ffmpeg-python

In [ ]:
import chardet
import json
import os
import subprocess

In [ ]:
import ffmpeg

In [ ]:
# Cutting using the json output we got above/before.

In [ ]:
def load_intervals_from_json(json_file):
    """Load word intervals from a JSON file."""
    with open(json_file, "r", encoding="utf-16") as f:
        data = json.load(f)

    # Convert dictionary to list of intervals
    intervals = [{"text": v[0], "xmin": v[1], "xmax": v[2]} for v in data.values()]
    return intervals

In [ ]:
def cut_video(video_file, output_folder, intervals):
    """Cut the video into multiple clips based on word timestamps."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, interval in enumerate(intervals):
        word = interval["text"]
        start_time = interval["xmin"] + 0.4
        end_time = interval["xmax"] + 1.2
        duration = end_time - start_time

        output_filename = f"{output_folder}/{i}_{word}.mp4"

        ffmpeg.input(video_file, ss=start_time, t=duration).output(output_filename).run(overwrite_output=True)

        # command = [
        #     "ffmpeg",
        #     "-i", video_file,             # Input video file
        #     "-ss", str(start_time),       # Start time
        #     "-t", str(duration),          # Duration
        #     "-c:v", "libx264",            # Video codec
        #     "-c:a", "aac",                # Audio codec
        #     "-y",                         # Overwrite without asking
        #     output_filename
        # ]

        # subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Saved: {output_filename}")

In [ ]:
# Example Usage
json_file = "parsed_textgrid.json" # "parsed_textgrid.json" # "example_grid.json"  # Replace with your JSON file
video_file = video_path # "kukbu.mp4" # "nags_sync_idk_MEHMETCAN_DUZELT_BURAYI.mp4"  # Replace with your actual video file
output_folder = "./drive/MyDrive/nags_word_videos" # "word_videos"

In [ ]:
# Load intervals from JSON
intervals = load_intervals_from_json(json_file)
# Cut video into videos that are number of words
cut_video(video_file, output_folder, intervals)

print("Video cutting complete!")

Saved: ./drive/MyDrive/nags_word_videos/0_kukbu.mp4
Saved: ./drive/MyDrive/nags_word_videos/1_kukbu.mp4
Saved: ./drive/MyDrive/nags_word_videos/2_vakit.mp4
Saved: ./drive/MyDrive/nags_word_videos/3_kufkak.mp4
Saved: ./drive/MyDrive/nags_word_videos/4_kakbu.mp4
Saved: ./drive/MyDrive/nags_word_videos/5_kafuka.mp4
Saved: ./drive/MyDrive/nags_word_videos/6_kafka.mp4
Saved: ./drive/MyDrive/nags_word_videos/7_kupku.mp4
Saved: ./drive/MyDrive/nags_word_videos/8_tapu.mp4
Saved: ./drive/MyDrive/nags_word_videos/9_ufuk.mp4
Saved: ./drive/MyDrive/nags_word_videos/10_kuvuk.mp4
Saved: ./drive/MyDrive/nags_word_videos/11_kabuk.mp4
Saved: ./drive/MyDrive/nags_word_videos/12_kukbak.mp4
Saved: ./drive/MyDrive/nags_word_videos/13_kufkak.mp4
Saved: ./drive/MyDrive/nags_word_videos/14_paket.mp4
Saved: ./drive/MyDrive/nags_word_videos/15_afkun.mp4
Saved: ./drive/MyDrive/nags_word_videos/16_bacak.mp4
Saved: ./drive/MyDrive/nags_word_videos/17_kavun.mp4
Saved: ./drive/MyDrive/nags_word_videos/18_kafkuk.mp4


# Testing the Off-by-one Errors

In [ ]:
# Off by one error testing.
# look at below
# it turns out, the code adds the frame after and before, i.e. don't worry of losing the last needed frame.
# The code always takes the first and last frame needed and doesn't miss it.

# when 0.0 - 1.0 length: 1000.99999
# when 0.0 - 0.999 length: 1000.99999
# when 0.0 - 0.982 length: 985
# when 0.005 - 0.982 length: 985
# when 0.017 - 0.982 length: 968

In [ ]:
intervals

[{'text': 'kukbu', 'xmin': 0.017, 'xmax': 0.982},
 {'text': 'kukbu', 'xmin': 1.83061161019806, 'xmax': 2.28977720936357}]

In [ ]:
import subprocess
import json

video_path = "./word_videos/0_kukbu.mp4"

def get_video_duration(video_path):
    cmd = [
        "ffprobe", "-v", "error",
        "-show_entries", "format=duration",
        "-of", "json", video_path
    ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    info = json.loads(result.stdout)
    return float(info["format"]["duration"]) * 1000  # Convert seconds to ms

duration_ms = get_video_duration(video_path)
print(f"Video duration (ms): {duration_ms}")


Video duration (ms): 968.0


In [ ]:
import subprocess
import json

video_path = "./word_videos/1_kukbu.mp4"

duration_ms = get_video_duration(video_path)
print(f"Video duration (ms): {duration_ms}")


Video duration (ms): 468.0
